### **Скрипт-инструкция к API отчетов Яндекс.Метрики для выгрузки отчетов в среду Python**

**Введение**

Настоящая скрипт-инструкция по обращению к API отчетов Яндекс.Метрики позволяет без использования интерфейса Яндекс.Метрики получать информацию о статистике посещений сайта и другие данные в формате Pandas DataFrame для работы в среде Python.

Справка Яндекс.Метрики о API Отчетов: https://yandex.ru/dev/metrika/doc/api2/api_v1/intro.html

Для работы с API отчетов Яндекс.Метрики в настоящей скрипт-инструкции используется библиотека **tapi_yandex_metrika**, документацию о которой можно рассмотреть по ссылке: https://github.com/pavelmaksimov/tapi-yandex-metrika

**Блок объявления и загрузки библиотек**

In [ ]:
!pip install tapi_yandex_metrika
import pandas as pd
from tapi_yandex_metrika import YandexMetrikaStats
import requests
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 278 kB 5.0 MB/s 
  Created wheel for tapi-yandex-metrika: filename=tapi_yandex_metrika-2022.4.8-py3-none-any.whl size=10719 sha256=5d029c4c193b962385d854c51b9f662b653962b74942b0befa41b235f29fd11f
  Stored in directory: /root/.cache/pip/wheels/7c/18/11/fc79b76c6cad2aee1cf6ae8d95d33c1297cedbb5d88d133e39
  Created wheel for tapi-wrapper2: filename=tapi_wrapper2-0.1.3-py3-none-any.whl size=8877 sha256=78429c0c91b7844ca507d17358063a2a35ff510cb83a321f7d09ebd0d4e2aa05
  Stored in directory: /root/.cache/pip/wheels/c4/64/55/179c1030f1babdd0f8063a74524947dea8ea6ea9d0b446da50
Successfully built tapi-yandex-metrika tapi-wrapper2


**Блок объявления токена API и номера счетчика Яндекс.Метрики**
_____________________________

In [ ]:
ACCESS_TOKEN = 'y0_*********B*************__*********p0yJpsUQ4' # Введите токен API
COUNTER_ID = '3*********' # Введите номер счетчика Яндекс.Метрики

Видео о том, как получить токен API (API_key) Яндекс Метрики: https://www.youtube.com/watch?v=_xCh8LHEJJA

*ВАЖНО: публикация токена API в открытый доступ запрещена, в противном случае третьи лица могут получить полный доступ к счетчику Яндекс.Метрики, в том числе к его настройкам и содержащимся в нем данным*

**Блок объявления параметров запроса к API отчетов Яндекс.Метрики**
_______________________________

**Группировка (dimension)** — это атрибут визита или хита, по которому можно сгруппировать данные.

В запросах к API группировки задаются в параметре dimensions. Если необходимо указать несколько группировок, перечислите их через запятую.

 

In [ ]:
dimensions = "ym:s:date,ym:s:TrafficSource,ym:s:UTMCampaign,ym:s:regionCity" # ,ym:s:UTMSource,ym:s:deviceCategory и т.д.

Список возможных группировок приведен на странице https://yandex.ru/dev/metrika/doc/api2/api_v1/intro.html в разделе "Группировки"

Также возможен отчет без группировок, в этом случае будет рассчитан суммарный результат.
_______________________________

**Метрика (metric)** — числовая величина, которая рассчитывается на основе атрибута хита или визита.

В запросах к API метрики задаются в параметре metrics. Если необходимо указать несколько метрик, перечислите их через запятую.

In [ ]:
metrics = "ym:s:visits,ym:s:users,ym:s:pageDepth" # ,ym:s:ecommercePurchases,ym:s:offlineCalls и т.д.

Список возможных метрик приведен на странице https://yandex.ru/dev/metrika/doc/api2/api_v1/intro.html в разделе "Метрики"

*Примечание: API отчетов поддерживает механизм задания параметров для группировок и метрик — параметризацию. Например, если необходимо получить количество целевых визитов, метрика ym:s:visits может быть параметризована следующим образом:* `ym:s:goal<goal_id>visits`

*Подробнее о параметризации группировок и метрик здесь: https://yandex.ru/dev/metrika/doc/api2/api_v1/param.html*
____________________________________

**Шаблоны (presets)** - необязательный параметр запроса к API отчетов, который позволяет выбирать готовый набор Группировок и Метрик

In [ ]:
preset = "traffic" # conversion, hourly, age и т.д.

В разделе "Шаблоны" по ссылке https://yandex.ru/dev/metrika/doc/api2/api_v1/presets/presets.html приведен полный список возможных шаблонов
______________________________________________________________

**Фильтры (filters)** - другой необязательный параметр, который позволяет выделить из данных отчета нужный нам сегмент

In [ ]:
filters = "ym:s:pageDepth>2"

*Примечание: метрика, по которой фильтруются данные, должна быть указана в параметре metrics*

Подробнее о порядке применения фильтров и возможных операторах фильтрации можно ознакомиться по ссылке:
https://yandex.ru/dev/metrika/doc/api2/api_v1/segmentation.html
_____________________________________________

Задаем временные границы, за которые необходимо получить данные 

In [ ]:
start_date = '2022-10-01'
end_date = '2022-10-02'

_____________________________________________
Список других возможных параметров приведен на странице https://yandex.ru/dev/metrika/doc/api2/api_v1/intro.html в разделе "Запросы к API"

**Блок выгрузки данных**
_________________________________

Инициализация параметров подключения к счетчику Яндекс.Метрики

In [ ]:
client = YandexMetrikaStats(access_token=ACCESS_TOKEN)

params = dict(
    ids = COUNTER_ID,
    date1 = start_date,
    date2 = end_date,
    metrics = metrics,
    dimensions = dimensions,
    accuracy = 'full', # точность вычисления результата (позволяет управлять семплированием)
    limit = 100000, # количество элементов на странице выдачи (максимально 100000 элементов)
    filters = filters
    #lang="ru" # Язык отчета 
    #preset = preset  
    #sort="ym:s:date" # Сортировка по столбцу
    # и т.д. (другие возможные параметры запроса)
)

Проверка инициализации параметров запроса

In [ ]:
params

Выгрузка данных из Яндекс.Метрики в Pandas DataFrame

In [ ]:
report = client.stats().get(params=params)

df = pd.DataFrame()

for page in report().pages(max_pages=100):
    dict_report = page().to_dicts()
    df1 = pd.DataFrame.from_dict(dict_report)
    df = df.append(df1)

Выводим получившийся Pandas DataFrame

In [ ]:
df

,ym:s:date,ym:s:trafficSource,ym:s:UTMCampaign,ym:s:regionCity,ym:s:visits,ym:s:users,ym:s:pageDepth
0,2022-10-02,Internal traffic,None,Moscow,2689.0,1894.0,2.098178
1,2022-10-01,Internal traffic,None,Moscow,2562.0,1840.0,2.165496
2,2022-10-02,Internal traffic,None,None,2454.0,1758.0,2.219641
3,2022-10-02,Direct traffic,None,None,2370.0,1878.0,2.652743
4,2022-10-01,Internal traffic,None,None,2092.0,1534.0,2.320746
...,...,...,...,...,...,...,...
6843,2022-10-02,Messenger traffic,allbe_free68_allclass_name--GR,Kaluga,1.0,1.0,1.000000
6844,2022-10-02,Messenger traffic,allbe_free68_allclass_name--GR,Orenburg,1.0,1.0,2.000000
6845,2022-10-02,Messenger traffic,allbe_zk_allclass_28-09-2022_name--sms_test_20.09,Yekaterinburg,1.0,1.0,24.000000
6846,2022-10-02,Messenger traffic,allbe_zk_allclass_28-09-2022_name--telegram_ka...,Vologda,1.0,1.0,1.000000
